In [1]:
from functions import *

In [2]:
# Read the CSV file into a DataFrame
result_dir = "../../exp_results"

In [3]:
summary_path = result_dir + "/result_summary.csv"
output_path = result_dir + "/correctness_per_dataset.csv"

check_correntness(summary_path, output_path)

    numTasks  utilization  TP  TN  FP  FN  accuracy
0        3.0          0.2  50   0   0   0      1.00
1        3.0          0.4  45   3   0   2      0.96
2        3.0          0.6  46   4   0   0      1.00
3        3.0          0.8  31  18   0   1      0.98
4        6.0          0.2  48   0   0   2      0.96
5        6.0          0.4  37   9   0   4      0.92
6        6.0          0.6  32  13   0   5      0.90
7        6.0          0.8  16  31   0   3      0.94
8        9.0          0.2  44   0   0   6      0.88
9        9.0          0.4  33   9   0   8      0.84
10       9.0          0.6  20  24   0   6      0.88
11       9.0          0.8   6  41   0   3      0.94
12      12.0          0.2  41   0   0   9      0.82
13      12.0          0.4  21  15   0  14      0.72
14      12.0          0.6  14  26   0  10      0.80
15      12.0          0.8   4  43   0   3      0.94
16      15.0          0.2  36   0   0  14      0.72
17      15.0          0.4  15  19   0  16      0.68
18      15.0

In [4]:
summary_path = result_dir + "/result_summary.csv"

num_cores = 1
num_tasks = 3
utilization =0.4 
detail_result_dir = result_dir + "/detail_result"

FN_subset_info = search_FN_info(summary_path, num_tasks, utilization)
print("FN_info")
print(FN_subset_info)

taskset_indice = FN_subset_info["tasksetIndex"].tolist()
for taskset_index in taskset_indice:
    print("\n\nTaskset_index: ", taskset_index)
    detail_result = show_detail_result(detail_result_dir, num_cores, num_tasks, utilization, taskset_index)
    print(detail_result)

FN_info
    nTasks  util  tasksetIndex  sched(Sim)  sched(proposed)
52       3   0.4             2        True            False
70       3   0.4            20        True            False


Taskset_index:  2
    WCRT(Sim)  sched(Sim)  WCRT(proposed)  sched(proposed)
id                                                        
1       74861        True           77469             True
2      132742        True          132742             True
3       92316        True               0            False


Taskset_index:  20
    WCRT(Sim)  sched(Sim)  WCRT(proposed)  sched(proposed)
id                                                        
1      316848        True          317124             True
2      147210        True          168275             True
3      185340        True               0            False


In [11]:
import pandas as pd
import os

def combine_detail_result(summary_path,  detail_result_dir, combine_detail_result_path):
    summary_df = pd.read_csv(summary_path, sep=",")
    
    results = []
    
    for _, result in summary_df.iterrows():
        if result['simulator_schedulability'] == True:
            num_cores = result['numCores']
            num_tasks = result['numTasks']
            utilization = result['utilization']
            taskset_index = result['tasksetIndex']
            
            filepath = os.path.join(detail_result_dir, str(num_cores) + 'cores', str(num_tasks) + 'tasks', str(utilization) + 'utilization')
            filename = str(num_cores) + 'cores_' + str(num_tasks) + 'tasks_' + str(utilization) + 'utilization_' + str(taskset_index) + '_result.csv'
            filepath_filename = os.path.join(filepath, filename)
            detail_df = pd.read_csv(filepath_filename, sep=",", header=None, index_col=0)
            detail_df = detail_df.iloc[1:] # remove the first row
            detail_df['numCores'] = num_cores
            detail_df['numTasks'] = num_tasks
            detail_df['utilization'] = utilization
            detail_df['tasksetIndex'] = taskset_index
            results.append(detail_df)
    
    
    # add column names
    combined_df = pd.concat(results, axis=0, ignore_index=True)
    combined_df.columns = ['name', 'WCRT(sim)', 'simulator_schedulability', 'WCRT(prop)', 'proposed_schedulability', 'numCores', 'numTasks', 'utilization', 'tasksetIndex']
    # sort the results by numTasks and utilization
    combined_df = combined_df.sort_values(['numTasks', 'utilization'])
    print(combined_df[:10])
    assert False
    
    combined_df.to_csv(combine_detail_result_path, index=False)
    
# Read the CSV file into a DataFrame
result_dir = "../../exp_results"
summary_path = result_dir + "/result_summary.csv"

# check conservativeness of results for each task
detail_result_dir = result_dir + "/detail_result"
combine_detail_result_path = detail_result_dir + "/combined_detail_result.csv"
combine_detail_result(summary_path,  detail_result_dir, combine_detail_result_path)

# output_path = result_dir + "/correctness_per_task.csv"

# check_correntness(combine_detail_result_path, output_path)

ValueError: Length mismatch: Expected axis has 5 elements, new values have 9 elements

In [7]:
## print results that WCRT(sim) > WCRT(proposed)
# print_non_conservative_results(combine_detail_result_path)

# assert that non convervative results are occured only when the system is not schedulable
check_non_conservative_results(summary_path, combine_detail_result_path)

check the correctness : 
If the system schedulability is true, the proposed method is always conservative 
